In [40]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"
# saved_mab_model = "checkpoints/mab_model_100.pth"
saved_mab_model = "checkpoints/mab_model_800.pth"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [41]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
# instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation."
instruction = "Analyze the sentiment of the following sentence and respond concisely."

# user_input = "I am extremely disappointed with the quality; it broke after just one day."
user_input = "The service at this restaurant was fantastic, and the staff were so friendly."
# user_input = "<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>"
# user_input = "I'm so happy with the product!"
# user_input = "The bright sunshine and gentle breeze made my afternoon truly delightful."
# user_input = "I felt deeply disappointed and frustrated after the meeting went completely off track."
# user_input = "Although the food at the restaurant was excellent, the service left much to be desired."


content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive. The use of words like "fantastic" and "friendly" convey a high level of satisfaction and appreciation for the service.<|eot_id|>


In [42]:
# get the probabilities 'positive', 'negative' and 'neutral'
inputs = tokenizer(gen_output, return_tensors='pt', add_special_tokens=False).to(device)
logits = llm.model(inputs['input_ids'], inputs['attention_mask']).logits
logits = logits[:, -3, :]
# print(logits)
# # get the probabilities of 'positive', 'negative' and 'neutral'
probs = torch.softmax(logits, dim=-1)
print("positive probability: ", probs[0, 31587])
print("negative probability: ", probs[0, 43324])
print("neutral probability: ", probs[0, 60668])
print("not clear probability: ", probs[0, 1962])


positive probability:  tensor(9.2385e-11, device='cuda:0', grad_fn=<SelectBackward0>)
negative probability:  tensor(1.4155e-10, device='cuda:0', grad_fn=<SelectBackward0>)
neutral probability:  tensor(2.8480e-12, device='cuda:0', grad_fn=<SelectBackward0>)
not clear probability:  tensor(5.8174e-12, device='cuda:0', grad_fn=<SelectBackward0>)


In [43]:
decoded_tokens = tokenizer.decode(inputs['input_ids'].tolist()[0])
print(decoded_tokens)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive. The use of words like "fantastic" and "friendly" convey a high level of satisfaction and appreciation for the service.<|eot_id|>


In [44]:
decoded_tokens


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nAnalyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>\n\nThe service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe sentiment of this sentence is overwhelmingly positive. The use of words like "fantastic" and "friendly" convey a high level of satisfaction and appreciation for the service.<|eot_id|>'

In [45]:
# Add this code to check the token IDs
print("Positive:", tokenizer.encode("positive"))
print("Negative:", tokenizer.encode("negative"))
print("Neutral:", tokenizer.encode("neutral"))
print("Not clear:", tokenizer.encode("not clear"))

Positive: [128000, 31587]
Negative: [128000, 43324]
Neutral: [128000, 60668]
Not clear: [128000, 1962, 2867]


In [46]:
inputs['input_ids'][0,-2]

tensor(13, device='cuda:0')

In [47]:
inputs['input_ids']

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
            271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
             13,    578,   1005,    315,   4339,   1093,    330,  61827,   5174,
              1,    323,    330,  82630,      1,  20599,    264,   1579,   2237,
            315,  24617,    323,  35996,    369,    279,   2532,     13, 128009]],
       device='cuda:0')

In [48]:
inputs

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
            271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
             13,    578,   1005,    315,   4339,   1093,    330,  61827,   5174,
              1,    323,    330,  82630,      1,  20599,    264,   1579,   2237,
            315,  24617,    323,  35996,    369,    279,   2532,     13, 128009]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [49]:
from llmexp.utils.data_utils import DataCollator
data_collator = DataCollator(tokenizer, max_length=512, instruction=instruction)

example = {
    'sentence': user_input,
    'label': 1
}

example = data_collator([example]).to(device)
print(example)


{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'context_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [50]:
print(tokenizer.decode(example['input_ids'][0] * example['attention_mask'][0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [51]:
gen_output = llm.generate(example['input_ids'], example['attention_mask'])
print(gen_output)

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
            271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
             11,  19392,    264,   1579,   2237,    315,  24617,    449,    279,
           2532,    323,   5687,     13, 128009]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        

In [52]:
gen_output['input_ids'][0]

tensor([128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
            25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
           220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
           279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
           285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
          2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
          5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
           271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
            11,  19392,    264,   1579,   2237,    315,  24617,    449,    279,
          2532,    323,   5687,     13, 128009], device='cuda:0')

In [53]:
tokenizer.decode(inputs['input_ids'][0,-1])

'<|eot_id|>'

In [54]:
# inputs['input_ids'][0] == gen_output['input_ids'][0]

In [55]:
# inputs['attention_mask'][0] == gen_output['attention_mask'][0]

In [56]:
# gen_output['attention_mask'][0]

In [57]:
# inputs['attention_mask'][0]
gen_output

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
              25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
             220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
             279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
             285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
            2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
            5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
             271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
              11,  19392,    264,   1579,   2237,    315,  24617,    449,    279,
            2532,    323,   5687,     13, 128009]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [58]:
logits = llm.model(gen_output['input_ids'][:,:-2], gen_output['attention_mask'][:,:-2]).logits
logits = logits[:, -1, :]
# # get the probabilities of 'positive', 'negative' and 'neutral'
probs = torch.softmax(logits, dim=-1)
print("positive probability: ", probs[0, 31587])
print("negative probability: ", probs[0, 43324])
print("neutral probability: ", probs[0, 60668])
print("not clear probability: ", probs[0, 1962])

positive probability:  tensor(4.4443e-10, device='cuda:0', grad_fn=<SelectBackward0>)
negative probability:  tensor(7.1664e-11, device='cuda:0', grad_fn=<SelectBackward0>)
neutral probability:  tensor(2.0237e-11, device='cuda:0', grad_fn=<SelectBackward0>)
not clear probability:  tensor(2.1396e-10, device='cuda:0', grad_fn=<SelectBackward0>)


In [59]:
print(tokenizer.decode(gen_output['input_ids'][0] * gen_output['attention_mask'][0]))
print(tokenizer.decode(inputs['input_ids'][0] * inputs['attention_mask'][0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive, indicating a high level of satisfaction with the service and staff.!
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive. The use of words like "fantastic" and "friend

In [60]:
print(tokenizer.decode(gen_output['input_ids'][0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive, indicating a high level of satisfaction with the service and staff.<|eot_id|>


In [79]:
from llmexp.trainer.mab_trainer import randomly_cut_and_pad_generations
cut_and_pad_gen_output = randomly_cut_and_pad_generations(example, gen_output, tokenizer)
print(cut_and_pad_gen_output)


{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
            271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
             11,  19392,    264]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], devi

In [80]:
print(tokenizer.decode(cut_and_pad_gen_output['input_ids'][0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive, indicating a


In [81]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm, hidden_size=1024)
mab_model.to(device)
print()

/tmp/1174450.1.gpu/ipykernel_1074858/3922162976.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), ll

In [82]:
input_ids = cut_and_pad_gen_output['input_ids']
attention_mask = cut_and_pad_gen_output['attention_mask']
logits, values = mab_model.get_logits_value(input_ids, attention_mask)


# mab_values = torch.softmax(logits, dim=-1)

context_mask = cut_and_pad_gen_output['context_mask']
# fill the 
masked_logits = logits.masked_fill(~context_mask[:,:-1].bool(), float('-inf'))
mab_values = torch.softmax(masked_logits, dim=-1)
# mab_values = dist.logits

In [83]:
mab_values.sum(-1)

tensor([1.], device='cuda:0', grad_fn=<SumBackward1>)

In [84]:
print(mab_values)
print(mab_values.shape)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.2959e-24, 5.1474e-23, 5.9863e-23, 3.4566e-22,
         2.4094e-21, 3.7357e-21, 6.3636e-21, 5.6688e-22, 3.3977e-20, 6.7397e-23,
         4.5766e-21, 1.3025e-22, 5.9851e-20, 1.9969e-19, 2.6321e-23, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.0051e-25,
         3.7526e-25, 6.4454e-27, 6.2222e-22, 7.9506e-18, 3.4136e-18, 2.5916e-21,
         1.9524e-14, 1.0000e

In [85]:
print(input_ids)
print(input_ids.shape)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
            271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
             11,  19392,    264]], device='cuda:0')
torch.Size([1, 75])


In [86]:
def visualize_tokens_with_values(input_ids, mab_values, context_mask, tokenizer):
    # Decode tokens one by one to preserve alignment
    tokens = []
    for i in range(input_ids.shape[1]):
        token = tokenizer.decode(input_ids[0, i:i+1])
        tokens.append(token)
    
    # Normalize MAB values to [0,1] for color intensity first
    mab_values = mab_values * context_mask[:,:-1]
    # Create a mask for non-zero values
    non_zero_mask = mab_values[0] != 0
    # normalized_values = (mab_values[0] - mab_values[0].min()) / (mab_values[0].max() - mab_values[0].min())
    normalized_values = torch.zeros_like(mab_values[0])
    # Only normalize non-zero values
    if non_zero_mask.any():  # Check if there are any non-zero values
        non_zero_values = mab_values[0][non_zero_mask]
        normalized_non_zero = (non_zero_values - non_zero_values.min()) / (non_zero_values.max() - non_zero_values.min())
        normalized_values[non_zero_mask] = normalized_non_zero
    
    # Pad normalized_values with a zero at the end
    padded_normalized_values = torch.cat([normalized_values, torch.zeros(1, device=mab_values.device)], dim=0)
    # Pad original mab_values with the last actual value
    padded_mab_values = torch.cat([mab_values[0], mab_values[0][-1:]], dim=0)
    
    # Generate HTML with colored text and values
    html_output = "<div style='font-family: monospace; line-height: 2; background-color: white; padding: 10px;'>"
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        # Use a gradient from white to green
        intensity = float(value)
        green_color = int(intensity * 200)  # Control the maximum intensity
        html_output += f'<span style="color: black; background-color: rgba(0, {green_color}, 0, 0.3); padding: 0.2em; margin: 0.1em; border-radius: 3px;" title="MAB: {orig_value:.3f}, Norm: {value:.3f}">{token}</span>'
    html_output += "</div>"
    
    # Print the values
    print("Token\tNormalized Value\tOriginal MAB Value")
    print("-" * 50)
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        print(f"{token}\t{value:.3f}\t\t{orig_value:.3f}")
    
    from IPython.display import HTML
    return HTML(html_output)

In [87]:
# Usage:
visualization = visualize_tokens_with_values(input_ids, mab_values, context_mask, tokenizer)
display(visualization)

Token	Normalized Value	Original MAB Value
--------------------------------------------------
<|begin_of_text|>	0.000		0.000
<|start_header_id|>	0.000		0.000
system	0.000		0.000
<|end_header_id|>	0.000		0.000


	0.000		0.000
Cut	0.000		0.000
ting	0.000		0.000
 Knowledge	0.000		0.000
 Date	0.000		0.000
:	0.000		0.000
 December	0.000		0.000
 	0.000		0.000
202	0.000		0.000
3	0.000		0.000

	0.000		0.000
Today	0.000		0.000
 Date	0.000		0.000
:	0.000		0.000
 	0.000		0.000
26	0.000		0.000
 Jul	0.000		0.000
 	0.000		0.000
202	0.000		0.000
4	0.000		0.000


	0.000		0.000
An	0.000		0.000
alyze	0.000		0.000
 the	0.000		0.000
 sentiment	0.000		0.000
 of	0.000		0.000
 the	0.000		0.000
 following	0.000		0.000
 sentence	0.000		0.000
 and	0.000		0.000
 respond	0.000		0.000
 conc	0.000		0.000
is	0.000		0.000
ely	0.000		0.000
.	0.000		0.000
<|eot_id|>	0.000		0.000
<|start_header_id|>	0.000		0.000
sentence	0.000		0.000
<|end_header_id|>	0.000		0.000


	0.000		0.000
The	0.000		0.000
 service	0.000		0.000
 a